In [22]:
from datasets import load_dataset, concatenate_datasets,Dataset,DatasetDict
import pandas as pd


In [3]:
naive_train_dataset = load_dataset('json', data_files='data/clue_json/guardian/naive_random/train.json', split='train')

naive_test_dataset = load_dataset('json', data_files='data/clue_json/guardian/naive_random/test.json', split='train')

naive_val_dataset = load_dataset('json', data_files='data/clue_json/guardian/naive_random/val.json', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
disjoint_train_dataset = load_dataset('json', data_files='data/clue_json/guardian/word_initial_disjoint/train.json',split='train')
disjoint_test_dataset = load_dataset('json', data_files='data/clue_json/guardian/word_initial_disjoint/test.json', split='train')
disjoint_val_dataset = load_dataset('json', data_files='data/clue_json/guardian/word_initial_disjoint/val.json', split='train')

In [7]:

concat_naive_dataset = concatenate_datasets([naive_train_dataset, naive_test_dataset, naive_val_dataset])
concat_disjoint_dataset = concatenate_datasets([disjoint_train_dataset, disjoint_test_dataset, disjoint_val_dataset])

concat_disjoint_train_val_dataset = concatenate_datasets([disjoint_train_dataset, disjoint_val_dataset])

In [ ]:
concat_naive_dataset = concat_naive_dataset.shuffle().remove_columns('idx')
concat_disjoint_train_val_dataset = concat_disjoint_train_val_dataset.shuffle().remove_columns('idx')


In [43]:
print(f'total length of naive dataste: {len(concat_naive_dataset)}')
print(f'total length of disjoint dataste: {len(concat_disjoint_dataset)}')

total length of naive dataste: 142380
total length of disjoint dataste: 142380


In [44]:
concat_naive_df = pd.DataFrame(concat_naive_dataset).drop_duplicates('target')
concat_disjoint_train_val_df= pd.DataFrame(concat_disjoint_train_val_dataset).drop_duplicates('target')
disjoint_test_df = pd.DataFrame(disjoint_test_dataset).drop_duplicates('target')



In [53]:

concat_half_disjoint_train_val_df = pd.DataFrame(concatenate_datasets([disjoint_train_dataset, disjoint_val_dataset])).groupby('target').sample(frac=0.51)
half_disjoint_test_df = pd.DataFrame(disjoint_test_dataset).groupby('target').sample(frac=0.51)

In [57]:
concat_half_disjoint_train_val_ds =  Dataset.from_pandas(concat_half_disjoint_train_val_df).rename_column ('input', 'clue').rename_column ('target', 'labels').shuffle()

half_disjoint_test_ds =  Dataset.from_pandas(half_disjoint_test_df).rename_column ('input', 'clue').rename_column ('target', 'labels').shuffle()

In [60]:
concat_half_disjoint_train_val_ds = concat_half_disjoint_train_val_ds.remove_columns('idx')

In [61]:
half_disjoint_ds = DatasetDict({'train':concat_half_disjoint_train_val_ds, 'test':half_disjoint_test_ds}).remove_columns('__index_level_0__')

In [46]:
concat_naive_ds = Dataset.from_pandas(concat_naive_df).rename_column ('input', 'clue').rename_column ('target', 'labels').shuffle()


concat_disjoint_train_val_ds = Dataset.from_pandas(concat_disjoint_train_val_df).rename_column ('input', 'clue').rename_column ('target', 'labels').shuffle()

disjoint_test_ds = Dataset.from_pandas(disjoint_test_df).rename_column ('input', 'clue').rename_column ('target', 'labels').shuffle()




In [47]:
disjoint_unique_ds = DatasetDict({'train':concat_disjoint_train_val_ds, 'test':disjoint_test_ds}).remove_columns('__index_level_0__')

In [48]:
naive_unique_ds = concat_naive_ds.train_test_split(test_size=0.15, shuffle=True).remove_columns('__index_level_0__')

In [62]:
half_disjoint_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'clue'],
        num_rows: 69339
    })
    test: Dataset({
        features: ['clue', 'labels'],
        num_rows: 21707
    })
})

In [49]:
disjoint_unique_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'clue'],
        num_rows: 42793
    })
    test: Dataset({
        features: ['clue', 'labels'],
        num_rows: 13495
    })
})

In [50]:
naive_unique_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'clue'],
        num_rows: 47844
    })
    test: Dataset({
        features: ['labels', 'clue'],
        num_rows: 8444
    })
})

In [63]:
# disjoint_unique_ds.push_to_hub('boda/word_init_disjoint_unique')
# naive_unique_ds.push_to_hub('boda/naive_random_unique')
half_disjoint_ds.push_to_hub('boda/word_init_disjoint_half')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/70 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/22 [00:00<?, ?ba/s]

In [64]:
############################ Sanity checks ############################

naive_unique = load_dataset('boda/naive_random_unique')
disjoint_unique = load_dataset('boda/word_init_disjoint_unique')
disjoint_half = load_dataset('boda/word_init_disjoint_half')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/47844 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8444 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/42793 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13495 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/69339 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21707 [00:00<?, ? examples/s]

In [100]:
############################ naive unique ############################
assert len(naive_unique['train']) +  len(naive_unique['test'])  == len(concat_naive_dataset.unique('target'))



############################ disjoint unique ############################
assert len(disjoint_unique['train']) == len(concat_disjoint_train_val_dataset.unique('target'))
assert len(disjoint_unique['test']) == len(disjoint_test_dataset.unique('target'))



In [105]:
############################## word init disjoint half ############################


disjoint_test_dataset_df = pd.DataFrame(concat_disjoint_dataset)
disjoint_half_df  =  pd.DataFrame(disjoint_half['train'])


In [106]:
sources = disjoint_half_df.value_counts('labels')
targets = disjoint_test_dataset_df.value_counts('target')

In [107]:
percentage = 0.0
for source in sources.index:
    percentage += sources[source]/targets[source]

percentage = percentage/len(disjoint_half_df)

In [108]:
percentage

0.4850919254446572